# Face Identification Using MTCNN
This notebook uses [MTCNN](https://github.com/ipazc/mtcnn) face detector.


## Prerequesites
Install the mtcnn library using
```
!pip install mtcnn
```


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import cv2
import tensorflow as tf
import numpy as np
from mtcnn import MTCNN
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import preprocessor
import configuration
import glob
import random
import time
import statistics
import viz

preproc = preprocessor.PreProcessor()
config = configuration.Configuration()
vizualizer = viz.Vizualizer()

2022-11-20 17:22:14.111698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def extract_face(model, image_file):
    image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
    

def extract_face_coords(model, image_file):
    pass

In [7]:
def infer(ax, model, image_file):
    # preparer = preprocessor.PreProcessor()
    # image = preparer.increase_brightness(cv2.imread(image_file), 100)
    # Sample prediction output:
    # {
    #     'box': [277, 90, 48, 63],
    #     'keypoints':
    #     {
    #         'nose': (303, 131),
    #         'mouth_right': (313, 141),
    #         'right_eye': (314, 114),
    #         'left_eye': (291, 117),
    #         'mouth_left': (296, 143)
    #     },
    #     'confidence': 0.99851983785629272
    # }
    
    image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
    
    start = time.process_time()
    predictions = model.detect_faces(image)
    time_taken = time.process_time() - start
    for prediction in predictions:
        bounding_box = prediction['box']
        confidence = prediction['confidence']
        if bounding_box:
            if confidence <= 0.33:
                edgecolor = 'yellow'
            elif confidence <= 0.66:
                edgecolor = 'orange'
            else:
                edgecolor = 'r'
            [x,y,w,h] = bounding_box
            rect = patches.Rectangle((x,y), w, h, linewidth=1, edgecolor=edgecolor, facecolor='none')
            ax.add_patch(rect)

        keypoints = prediction['keypoints']
        if keypoints:
            for point in keypoints.values():
                ax.plot(point[0], point[1], marker="x")
        
    ax.imshow(image, aspect='auto')
    # ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return time_taken
    

In [9]:
model = MTCNN()
times = []
for cur_class in range(len(config.class_dict)):
    new_times = vizualizer.infer_and_plot(model, infer, 
                                          config.class_dict[cur_class], f'{config.IMAGES_BASE}/c{cur_class}', 
                                          num_images=1, cols=1) #, plt_width=14, plt_height=32)
    times = [*times, *new_times]

print(f'Average time for inference:{statistics.mean(times)} seconds')
print(f'Anticipated preprocessing time for training dataset of 22,000:{22000 * statistics.mean(times) / 60} mins')


TypeError: __init__() got an unexpected keyword argument 'device'